In [12]:
import json

# 读取原始数据
input_file = '/mnt/workspace/chat-huan/chat-Yu/extract-dialogue/qianfu.jsonl'
output_file = '/mnt/workspace/chat-huan/chat-Yu/extract-dialogue/output.jsonl'

with open(input_file, 'r', encoding='utf-8') as f:
    lines = [json.loads(line.strip()) for line in f if line.strip()]

# 遍历并提取所需段落
result = []
for i in range(1, len(lines)):
    if lines[i]['role'] == '余则成':
        result.append(lines[i - 1])  # 添加前一个段落
        result.append(lines[i])      # 添加当前“余则成”的段落

# 去重（可选，如果怕重复）
seen = set()
unique_result = []
for item in result:
    key = json.dumps(item, ensure_ascii=False)
    if key not in seen:
        seen.add(key)
        unique_result.append(item)

# 写入输出文件
with open(output_file, 'w', encoding='utf-8') as f:
    for item in unique_result:
        f.write(json.dumps(item, ensure_ascii=False) + '\n')


In [19]:
!pip install dashscope

Looking in indexes: https://mirrors.cloud.aliyuncs.com/pypi/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 47.1 MB/s eta 0:00:00
DEPRECATION: pytorch-lightning 1.7.7 has a non-standard dependency specifier torch>=1.9.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 23.3.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [25]:
import os
import dashscope
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor

# 配置参数
DASHSCOPE_API_KEY = "sk-5055b11bf7dd4181a5c7adc4353175df"
TARGET_COUNT = 500
BATCH_SIZES = [50, 100, 200, 150, 16]  # 分阶段生成数量
EXAMPLE_DATA = """
{"role": "甲", "dialogue": "是呀，前不久..."}
{"role": "余则成", "dialogue": "你迟到了..."}
...（你的完整示例数据）
"""

# 设置 DashScope API Key
os.environ['DASHSCOPE_API_KEY'] = DASHSCOPE_API_KEY

def generate_dialogues(prompt, batch_size):
    """调用 DashScope API 生成对话数据"""
    messages = [
        {
            "role": "system",
            "content": "你是一个1940年代谍战剧编剧助手，需要生成符合以下要求的对话数据：\n"
                       "- 角色：余则成/张名义/吕宗方/其他军统人物\n"
                       "- 内容：情报交接/密码破译/内部审查/战略分析\n"
                       "- 风格：使用电报术语（如'总部电讯处'）和军事暗语（如'夜莺已激活'）\n"
                       "- 格式：严格每行一个{\"role\":\"...\",\"dialogue\":\"...\"}的JSONL"
        },
        {
            "role": "user",
            "content": f"参考示例：\n{EXAMPLE_DATA}\n请生成{batch_size}条同类对话"
        }
    ]

    try:
        response = dashscope.Generation.call(
            api_key=os.getenv('DASHSCOPE_API_KEY'),
            model="deepseek-v3",  # 您可以根据需要更换模型名称
            messages=messages,
            result_format="message",
            max_tokens=1000,  # 设置最大 token 数量限制为 1000
            temperature=0.7
        )

        return response.output.choices[0].message.content
    except Exception as e:
        print(f"API调用失败: {str(e)}")
        return None

def validate_entry(line):
    """验证并清洗数据条目"""
    try:
        entry = json.loads(line.strip())
        if entry.get("role") and entry.get("dialogue"):
            return json.dumps({"role": entry["role"], "dialogue": entry["dialogue"]}, ensure_ascii=False)
    except:
        return None

def process_batch(batch_size, collected_data):
    """处理每个批次，调用API并清洗数据"""
    raw_data = generate_dialogues(EXAMPLE_DATA, batch_size)
    
    if raw_data:
        valid_count = 0
        for line in raw_data.split('\n'):
            if cleaned := validate_entry(line):
                collected_data.append(cleaned)
                valid_count += 1
        return valid_count
    return 0

def main():
    collected_data = []
    progress_bar = tqdm(total=TARGET_COUNT, desc="生成进度")
    
    # 使用 ThreadPoolExecutor 实现并行调用
    with ThreadPoolExecutor(max_workers=5) as executor:
        futures = []
        for idx, batch_size in enumerate(BATCH_SIZES, 1):
            # 动态调整生成参数
            current_needed = TARGET_COUNT - len(collected_data)
            actual_batch = min(batch_size, current_needed)
            
            print(f"\n[阶段 {idx}] 正在生成 {actual_batch} 条...")

            # 异步提交每个批次的任务
            futures.append(executor.submit(process_batch, actual_batch, collected_data))

        # 等待所有任务完成并更新进度
        for future in futures:
            valid_count = future.result()
            progress_bar.update(valid_count)
            print(f"有效条目：{valid_count}/{TARGET_COUNT}")

            # 达到目标立即停止
            if len(collected_data) >= TARGET_COUNT:
                break

    # 保存结果
    with open("enhanced_data.jsonl", "w", encoding="utf-8") as f:
        f.write('\n'.join(collected_data[:TARGET_COUNT]))

    print(f"\n生成完成！最终数量：{len(collected_data)}条")

if __name__ == "__main__":
    main()



生成进度:   0%|          | 0/500 [00:00<?, ?it/s]


[阶段 1] 正在生成 50 条...

[阶段 2] 正在生成 100 条...

[阶段 3] 正在生成 200 条...

[阶段 4] 正在生成 150 条...

[阶段 5] 正在生成 16 条...



生成进度:   8%|▊         | 40/500 [02:19<26:43,  3.49s/it]

有效条目：40/500
有效条目：42/500



生成进度:  24%|██▍       | 122/500 [02:25<05:55,  1.06it/s]

有效条目：40/500



生成进度:  28%|██▊       | 138/500 [03:01<07:55,  1.31s/it]

API调用失败: 'NoneType' object has no attribute 'choices'
有效条目：0/500
有效条目：16/500

生成完成！最终数量：138条


In [28]:
import json

def convert_for_instruction_output(input_file, output_file):
    with open(input_file, 'r', encoding='utf-8') as infile:
        lines = [json.loads(line.strip()) for line in infile if line.strip()]
    
    converted = []
    for i in range(1, len(lines)):
        current = lines[i]
        prev = lines[i - 1]

        # 如果当前行是余则成说的，就用上一行当 instruction
        if current['role'] == '余则成':
            item = {
                "instruction": prev["dialogue"],
                "input": "",
                "output": current["dialogue"]
            }
            converted.append(item)

    # 写入文件
    with open(output_file, 'w', encoding='utf-8') as outfile:
        for item in converted:
            outfile.write(json.dumps(item, ensure_ascii=False) + '\n')

    print(f"转换完成，共生成 {len(converted)} 条对话，保存至 {output_file}")

# 使用方式
convert_for_instruction_output('enhanced_data.jsonl', 'converted_instruction_output.jsonl')


转换完成，共生成 51 条对话，保存至 converted_instruction_output.jsonl


In [30]:
%cd  /mnt/workspace/chat-huan/chat-Yu/

/mnt/workspace/chat-huan/chat-Yu


In [32]:
import torch
from modelscope import snapshot_download, AutoModel, AutoTokenizer
import os

model_dir = snapshot_download('Qwen/Qwen2.5-7B-Instruct', cache_dir='.', revision='master')


Downloading:   0%|          | 0.00/663 [00:00<?, ?B/s]
Downloading: 100%|██████████| 663/663 [00:00<00:00, 1.38kB/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]
Downloading: 100%|██████████| 2.00/2.00 [00:00<00:00, 3.99B/s]

Downloading:   0%|          | 0.00/243 [00:00<?, ?B/s]
Downloading: 100%|██████████| 243/243 [00:00<00:00, 517B/s]

Downloading:   0%|          | 0.00/11.1k [00:00<?, ?B/s]
Downloading: 100%|██████████| 11.1k/11.1k [00:00<00:00, 23.2kB/s]

Downloading:   0%|          | 0.00/1.59M [00:00<?, ?B/s]
Downloading: 100%|██████████| 1.59M/1.59M [00:00<00:00, 2.58MB/s]

Downloading:   0%|          | 0.00/3.67G [00:00<?, ?B/s]
Downloading:   0%|          | 1.00M/3.67G [00:00<34:49, 1.89MB/s]
Downloading:   1%|          | 19.0M/3.67G [00:00<01:38, 40.0MB/s]
Downloading:   2%|▏         | 63.0M/3.67G [00:00<00:28, 135MB/s] 
Downloading:   3%|▎         | 111M/3.67G [00:00<00:13, 294MB/s] 
Downloading:   3%|▎         | 111M/3.67G [00:00<00:13, 294MB/s]
Downloading: 

In [33]:
import json

def jsonl_to_json(jsonl_path, json_path):
    with open(jsonl_path, 'r', encoding='utf-8') as infile:
        data = [json.loads(line) for line in infile if line.strip()]

    with open(json_path, 'w', encoding='utf-8') as outfile:
        json.dump(data, outfile, ensure_ascii=False, indent=2)

    print(f"转换完成，共 {len(data)} 条数据，已保存为 {json_path}")

# 使用方式
jsonl_to_json('yu.jsonl', 'yu.json')


转换完成，共 51 条数据，已保存为 yu.json


In [35]:
from datasets import Dataset
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer, GenerationConfig
from peft import LoraConfig, TaskType, get_peft_model


def process_func(example):
    MAX_LENGTH = 384    # Llama分词器会将一个中文字切分为多个token，因此需要放开一些最大长度，保证数据的完整性
    input_ids, attention_mask, labels = [], [], []
    instruction = tokenizer(f"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 Jul 2024\n\n现在你要扮演皇帝身边的女人--甄嬛<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n{example['instruction'] + example['input']}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n", add_special_tokens=False)  # add_special_tokens 不在开头加 special_tokens
    response = tokenizer(f"{example['output']}<|eot_id|>", add_special_tokens=False)
    input_ids = instruction["input_ids"] + response["input_ids"] + [tokenizer.pad_token_id]
    attention_mask = instruction["attention_mask"] + response["attention_mask"] + [1]  # 因为eos token咱们也是要关注的所以 补充为1
    labels = [-100] * len(instruction["input_ids"]) + response["input_ids"] + [tokenizer.pad_token_id]  
    if len(input_ids) > MAX_LENGTH:  # 做一个截断
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

if __name__ == "__main__":
    model = AutoModelForCausalLM.from_pretrained('./Qwen/Qwen2___5-7B-Instruct', device_map="auto",torch_dtype=torch.bfloat16)
    model.enable_input_require_grads() # 开启梯度检查点时，要执行该方法
    tokenizer = AutoTokenizer.from_pretrained('./Qwen/Qwen2___5-7B-Instruct', use_fast=False, trust_remote_code=True)
    tokenizer.pad_token = tokenizer.eos_token

    # 将JSON文件转换为CSV文件
    df = pd.read_json('yu.json')
    ds = Dataset.from_pandas(df)
    tokenized_id = ds.map(process_func, remove_columns=ds.column_names)

    config = LoraConfig(
        task_type=TaskType.CAUSAL_LM, 
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
        inference_mode=False, # 训练模式
        r=8, # Lora 秩
        lora_alpha=32, # Lora alaph，具体作用参见 Lora 原理
        lora_dropout=0.1# Dropout 比例
    )
    model = get_peft_model(model, config)
    model.print_trainable_parameters() # 打印总训练参数

    args = TrainingArguments(
        output_dir="./output/Qwen2___5-7B-Instruct_lora",
        per_device_train_batch_size=4,
        gradient_accumulation_steps=4,
        logging_steps=10,
        num_train_epochs=3,
        save_steps=100, # 为了快速演示，这里设置10，建议你设置成100
        learning_rate=1e-4,
        save_on_each_node=True,
        gradient_checkpointing=True
    )
    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=tokenized_id,
        data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
    )
    trainer.train() # 开始训练 
    # 在训练参数中设置了自动保存策略此处并不需要手动保存。

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Map:   0%|          | 0/51 [00:00<?, ? examples/s]

trainable params: 20,185,088 || all params: 7,635,801,600 || trainable%: 0.2643
[2025-04-14 16:54:55,351] [INFO] [real_accelerator.py:222:get_accelerator] Setting ds_accelerator to cuda (auto detect)


df: /root/.triton/autotune: 没有那个文件或目录
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/usr/local/lib/python3.11/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss


/usr/local/lib/python3.11/site-packages/peft/utils/save_and_load.py:195: UserWarning: Could not find a config file in ./Qwen/Qwen2___5-7B-Instruct - will assume that the vocabulary was not modified.
  warnings.warn(
